In [10]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import parse_kraken_orderbook
# from hydradx.model.amm.arbitrage_agent_new import get_arb_swaps, calculate_profit, execute_arb
from hydradx.model.amm.centralized_market import CentralizedMarket, OrderBook
import hydradx.model.amm.arbitrage_agent as arb
import hydradx.model.amm.arbitrage_agent_new as arb2

from hydradxapi import HydraDX
from pprint import pprint
import requests



import json, os


In [11]:

# TODO move to config file
arb_list = [
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 10), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 23), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 2), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 7), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 18), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 21), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 22), 'order_book': ('HDX','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 10), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 23), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 2), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 7), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 18), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 21), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 22), 'order_book': ('DOT','USD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 10), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 23), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 2), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 7), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 18), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 21), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 22), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 10), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 23), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 2), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 7), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 18), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 21), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 22), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('DOT', 'WETH'), 'tkn_ids': (5, 4), 'order_book': ('DOT','ETH')},
    {'tkns': ('DOT', 'WETH'), 'tkn_ids': (5, 20), 'order_book': ('DOT','ETH')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 10), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 23), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 2), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 7), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 18), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 21), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 22), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 10), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 23), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 2), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 7), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 18), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 21), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 22), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 10), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 23), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 2), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 7), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 18), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 21), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 22), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('DOT', 'WBTC'), 'tkn_ids': (5, 3), 'order_book': ('DOT','XBT')},
    {'tkns': ('DOT', 'WBTC'), 'tkn_ids': (5, 19), 'order_book': ('DOT','XBT')},
    {'tkns': ('DOT', 'IBTC'), 'tkn_ids': (5, 11), 'order_book': ('DOT','XBT')},
    # {'tkns': ('WETH', 'WBTC'), 'tkn_ids': (4, 3), 'order_book': ('XETH','XXBT')},
    {'tkns': ('WETH', 'WBTC'), 'tkn_ids': (4, 19), 'order_book': ('XETH','XXBT')},
    {'tkns': ('WETH', 'IBTC'), 'tkn_ids': (4, 11), 'order_book': ('XETH','XXBT')},
    # {'tkns': ('WETH', 'WBTC'), 'tkn_ids': (20, 3), 'order_book': ('XETH','XXBT')},
    {'tkns': ('WETH', 'WBTC'), 'tkn_ids': (20, 19), 'order_book': ('XETH','XXBT')},
    {'tkns': ('WETH', 'IBTC'), 'tkn_ids': (20, 11), 'order_book': ('XETH','XXBT')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 10), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 23), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 2), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 7), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 18), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 21), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 22), 'order_book': ('ASTR','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 10), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 23), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 2), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 7), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 18), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 21), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 22), 'order_book': ('CFG','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 10), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 23), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 2), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 7), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 18), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 21), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 22), 'order_book': ('BNC','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 10), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 23), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 2), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 7), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 18), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 21), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 22), 'order_book': ('GLMR','USD')},
]

order_books = {}
for arb_cfg in arb_list:
    tkn_pair = arb_cfg['order_book']
    if tkn_pair not in order_books:
        resp = requests.get('https://api.kraken.com/0/public/Depth?pair=' + tkn_pair[0] + tkn_pair[1])
        y = resp.json()
        order_books[tkn_pair] = y['result'][tkn_pair[0] + tkn_pair[1]]
    


In [12]:
def get_unique_name(ls: list[str], name: str) -> str:
    if name not in ls:
        return name
    else:
        c = 1
        while name + str(c).zfill(3) in ls:
            c += 1
        return name + str(c).zfill(3)

In [13]:
RPC="wss://hydradx-rpc.dwellir.com"

chain = HydraDX(RPC) 
chain.connect()

# Omnipool metadata
# TODO: move to JSON file
metadata = [
    {'symbol': 'HDX', 'id': 0, 'decimals': 12},
    {'symbol': 'LRNA', 'id': 1, 'decimals': 12},
    {'symbol': 'DAI', 'id': 2, 'decimals': 18},
    {'symbol': 'WBTC', 'id': 3, 'decimals': 8},
    {'symbol': 'WETH', 'id': 4, 'decimals': 18},
    {'symbol': 'DOT', 'id': 5, 'decimals': 10},
    {'symbol': 'APE', 'id': 6, 'decimals': 18},
    {'symbol': 'USDC', 'id': 7, 'decimals': 6},
    {'symbol': 'PHA', 'id': 8, 'decimals': 12},
    {'symbol': 'ASTR', 'id': 9, 'decimals': 18},
    {'symbol': 'USDT', 'id': 10, 'decimals': 6},
    {'symbol': 'IBTC', 'id': 11, 'decimals': 8},
    {'symbol': 'ZTG', 'id': 12, 'decimals': 10},
    {'symbol': 'CFG', 'id': 13, 'decimals': 18},
    {'symbol': 'BNC', 'id': 14, 'decimals': 12},
    {'symbol': 'vDOT', 'id': 15, 'decimals': 10},
    {'symbol': 'GLMR', 'id': 16, 'decimals': 18},
    {'symbol': 'INTR', 'id': 17, 'decimals': 10},
    {'symbol': 'DAI', 'id': 18, 'decimals': 18},
    {'symbol': 'WBTC', 'id': 19, 'decimals': 8},
    {'symbol': 'WETH', 'id': 20, 'decimals': 18},
    {'symbol': 'USDC', 'id': 21, 'decimals': 6},
    {'symbol': 'USDC', 'id': 22, 'decimals': 6},
    {'symbol': 'USDT', 'id': 23, 'decimals': 6},
    ]

asset_list = []
fees = {}
tokens = {}
asset_map = {}

for i in range(len(metadata)):
    try:
        state = chain.api.omnipool.asset_state(metadata[i]['id'])
    except:
        continue
        
    fee = chain.api.fees.asset_fees(metadata[i]['id'])

    tkn = get_unique_name(asset_list, metadata[i]['symbol'])
    asset_list.append(tkn)
    asset_map[i] = tkn
    tokens[tkn] = {
        'liquidity': state.reserve / 10**metadata[i]['decimals'],
        'LRNA': state.hub_reserve / 10**12
    }
    fees[tkn] = {"asset_fee": fee.asset_fee / 100, "protocol_fee": fee.protocol_fee / 100}
        
chain.close()

pprint(tokens)


{'ASTR': {'LRNA': 9100.991132122079, 'liquidity': 4603801.100324849},
 'BNC': {'LRNA': 6984.981129444166, 'liquidity': 721332.878917896},
 'CFG': {'LRNA': 8869.201612256744, 'liquidity': 601012.8716761052},
 'DAI': {'LRNA': 13115.950197480224, 'liquidity': 332743.9021445043},
 'DAI001': {'LRNA': 31752.634079689127, 'liquidity': 801710.7711294643},
 'DOT': {'LRNA': 70044.21991952446, 'liquidity': 428259.7038744445},
 'GLMR': {'LRNA': 5280.000925885971, 'liquidity': 673508.617803954},
 'HDX': {'LRNA': 19549.13037691755, 'liquidity': 117695971.7149292},
 'IBTC': {'LRNA': 34949.26083262988, 'liquidity': 25.8979437},
 'USDT': {'LRNA': 85174.15187697306, 'liquidity': 2155620.860359},
 'WBTC': {'LRNA': 50.601798806111, 'liquidity': 0.04374934},
 'WBTC001': {'LRNA': 41825.898662570326, 'liquidity': 31.07199565},
 'WETH': {'LRNA': 21682.12463858625, 'liquidity': 307.36181823539135},
 'WETH001': {'LRNA': 71561.87148593721, 'liquidity': 1017.5153743142145},
 'ZTG': {'LRNA': 4538.957735895216, 'li

In [14]:

ob_objs = {}
for tkn_pair in order_books:
    ob = parse_kraken_orderbook(order_books[tkn_pair])
    ob_obj = OrderBook(
        bids=[[bid['price'], bid['amount']] for bid in ob['bids']],
        asks=[[ask['price'], ask['amount']] for ask in ob['asks']]
    )
    ob_objs[tkn_pair] = ob_obj
    
order_book_asset_list = []
for tkn_pair in order_books:
    for tkn in tkn_pair:
        if tkn not in order_book_asset_list:
            order_book_asset_list.append(tkn)

cex_fee = 0.0016
buffer = 0.0002

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}

# lrna_fee = 0.0
# asset_fee = 0.0
# cex_fee = 0.0
# buffer = 0.0


cex = CentralizedMarket(
    order_book=ob_objs,
    asset_list=order_book_asset_list,
    trade_fee=cex_fee
)

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)

init_state = op_state.copy()


In [15]:
order_book_map = {}
for i in range(len(arb_list)):
    base_id, quote_id = arb_list[i]['tkn_ids']
    orderbook_tkn_pair = arb_list[i]['order_book']
    if base_id in asset_map and quote_id in asset_map:
        tkn_pair = (asset_map[base_id], asset_map[quote_id])
        order_book_map[tkn_pair] = orderbook_tkn_pair 
    
pprint(order_book_map)

{('ASTR', 'DAI'): ('ASTR', 'USD'),
 ('ASTR', 'DAI001'): ('ASTR', 'USD'),
 ('ASTR', 'USDT'): ('ASTR', 'USD'),
 ('BNC', 'DAI'): ('BNC', 'USD'),
 ('BNC', 'DAI001'): ('BNC', 'USD'),
 ('BNC', 'USDT'): ('BNC', 'USD'),
 ('CFG', 'DAI'): ('CFG', 'USD'),
 ('CFG', 'DAI001'): ('CFG', 'USD'),
 ('CFG', 'USDT'): ('CFG', 'USD'),
 ('DOT', 'DAI'): ('DOT', 'USD'),
 ('DOT', 'DAI001'): ('DOT', 'USD'),
 ('DOT', 'IBTC'): ('DOT', 'XBT'),
 ('DOT', 'USDT'): ('DOT', 'USD'),
 ('DOT', 'WBTC001'): ('DOT', 'XBT'),
 ('DOT', 'WETH'): ('DOT', 'ETH'),
 ('DOT', 'WETH001'): ('DOT', 'ETH'),
 ('GLMR', 'DAI'): ('GLMR', 'USD'),
 ('GLMR', 'DAI001'): ('GLMR', 'USD'),
 ('GLMR', 'USDT'): ('GLMR', 'USD'),
 ('HDX', 'DAI'): ('HDX', 'USD'),
 ('HDX', 'DAI001'): ('HDX', 'USD'),
 ('HDX', 'USDT'): ('HDX', 'USD'),
 ('IBTC', 'DAI'): ('XXBT', 'ZUSD'),
 ('IBTC', 'DAI001'): ('XXBT', 'ZUSD'),
 ('IBTC', 'USDT'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'DAI'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'DAI001'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'USDT'): ('XXBT', 'ZUS

In [16]:
cex1 = cex.copy()
op_state1 = op_state.copy()
all_swaps1 = arb.get_arb_swaps(op_state1, cex1, order_book_map, buffer=buffer)
tot_swaps = 0
for tkn_pair in all_swaps1:
    if all_swaps1[tkn_pair] != []:
        print(tkn_pair)
        print(all_swaps1[tkn_pair])
        tot_swaps += len(all_swaps1[tkn_pair])
print(tot_swaps)

cex2 = cex.copy()
op_state2 = op_state.copy()
all_swaps2 = arb2.get_arb_swaps(op_state2, cex2, order_book_map, buffer=buffer)
print("strategy 2")
pprint(all_swaps2)
pprint(len(all_swaps2))

('WETH', 'USDT')
[{'dex': {'trade': 'sell', 'buy_asset': 'USDT', 'sell_asset': 'WETH', 'price': 1772.84, 'amount': 0.2543827629089355, 'min_buy': 452.2783252595186}, 'cex': {'trade': 'buy', 'buy_asset': 'XETH', 'sell_asset': 'ZUSD', 'price': 1773.0172839999998, 'amount': 0.2543827629089355}}]
('WETH', 'DAI')
[{'dex': {'trade': 'sell', 'buy_asset': 'DAI', 'sell_asset': 'WETH', 'price': 1772.84, 'amount': 0.17244561195373537, 'min_buy': 306.6925290739417}, 'cex': {'trade': 'buy', 'buy_asset': 'XETH', 'sell_asset': 'ZUSD', 'price': 1773.0172839999998, 'amount': 0.17244561195373537}}]
('WBTC001', 'DAI')
[{'dex': {'trade': 'sell', 'buy_asset': 'DAI', 'sell_asset': 'WBTC001', 'price': 33913.3, 'amount': 0.001104492187500001, 'min_buy': 344.218744577229}, 'cex': {'trade': 'buy', 'buy_asset': 'XXBT', 'sell_asset': 'ZUSD', 'price': 33916.69133, 'amount': 0.001104492187500001}}]
('IBTC', 'USDT')
[{'dex': {'trade': 'sell', 'buy_asset': 'USDT', 'sell_asset': 'IBTC', 'price': 33913.3, 'amount': 0.0

In [17]:
state1 = init_state.copy()
state2 = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list + order_book_asset_list}
init_agent = Agent(holdings=holdings, unique_id='bot')
agent1 = init_agent.copy()
agent2 = init_agent.copy()
cex1 = cex.copy()
cex2 = cex.copy()

arb.execute_arb(state1, cex1, agent1, all_swaps1)
arb2.execute_arb(state2, cex2, agent2, all_swaps2)

asset_map = {'WETH': 'ETH',
             'XETH': 'ETH',
             'XXBT': 'BTC',
             'WBTC': 'BTC',
             'ZUSD': 'USD',
             'USDT': 'USD',
             'USDC': 'USD',
             'DAI': 'USD',
             'USDT001': 'USD',
             'DAI001': 'USD',
             'WETH001': 'ETH',
             'WBTC001': 'BTC',
             'IBTC': 'BTC',
             'XBT': 'BTC',}

profit1 = arb.calculate_profit(init_agent, agent1, asset_map)
profit2 = arb2.calculate_profit(init_agent, agent2, asset_map)

asset_map_omnipool = {
    'ETH': 'WETH',
    'XETH': 'WETH',
    'XXBT': 'WBTC',
    'BTC': 'WBTC',
    'ZUSD': 'USDT',
    'USD': 'USDT',
    'USDC': 'USDT',
    'DAI': 'USDT',
    'USDT001': 'USDT',
    'DAI001': 'USDT',
    'WETH001': 'WETH',
    'WBTC001': 'WBTC',
    'IBTC': 'WBTC',
    'XBT': 'WBTC',
}


profit1_omnipool = arb.calculate_profit(init_agent, agent1, asset_map_omnipool)
profit2_omnipool = arb2.calculate_profit(init_agent, agent2, asset_map_omnipool)

print("strategy 1 profit")
print(profit1)
print("strategy 1 profit at Omnipool spot")
print(sum([profit1_omnipool[asset] * state1.price(state1, asset, 'USDT') for asset in profit1_omnipool]))
print("strategy 2 profit")
print(profit2)
print("strategy 2 profit at Omnipool spot")
print(sum([profit2_omnipool[asset] * state2.price(state2, asset, 'USDT') for asset in profit2_omnipool]))
            

strategy 1 profit
{'HDX': 0, 'USD': 2.0176838636398315, 'BTC': 2.7418136596679688e-06, 'ETH': 0.0, 'DOT': 0.0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0, 'BNC': 0, 'vDOT': 0, 'GLMR': 0.0}
strategy 1 profit at Omnipool spot
2.0978642297663552
strategy 2 profit
{'HDX': 0, 'USD': 1.9377021789550781, 'BTC': 9.5367431640625e-06, 'ETH': 0.0, 'DOT': 0.0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0, 'BNC': 0, 'vDOT': 0, 'GLMR': 0.0}
strategy 2 profit at Omnipool spot
2.2167486849842923
